In [ ]:
import numpy as np

from bikewheelcalc import BicycleWheel, Rim, Hub, Spoke
import bikewheellib as bl

import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

In [ ]:
R = 0.300
GJ = 25.0
EI1 = 100.
EI2 = GJ / 0.2

w = BicycleWheel()
w.hub = Hub(diam1=0.05, width1=0.025, width2=0.025)
w.rim = Rim(radius=R, area=100e-6,
            I11=GJ / 26.0e9, I22=EI2 / 26.0e9, I33=EI1 / 26.0e9, Iw=0.0 / 69.0e9,
            young_mod=69.0e9, shear_mod=26.0e9)

In [ ]:
# Radial stiffness vs. Pippard Number
ns_0 = 36.
ds_0 = 1.8e-3

n_spk = np.arange(6, 37, 2)

K_rad_sm = np.zeros(n_spk.shape)
K_rad_ds = np.zeros(n_spk.shape)
n_SP = np.zeros(n_spk.shape)  # Smith-Pippard number

for i, ns in enumerate(n_spk):
    
    # Calculate spoke diameter to keep ns*As constant
    ds = ds_0 * np.sqrt(ns_0 / ns)
    w.lace_radial(n_spokes=int(ns), diameter=ds, young_mod=200e9, offset=0.00)
    w.apply_tension(0.01)
    
    rr = bl.ModeMatrix(w, N=4*len(w.spokes))
    
    B_v0 = rr.B_theta(0., comps=[1])
                       
    K_sm = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)
    K_ds = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=False)
    
    F_ext = rr.calc_F_ext([0.], np.array([[0., 1., 0., 0.]]))

    K_rad_sm[i] = 1./(B_v0.dot(np.linalg.solve(K_sm, F_ext)))
    K_rad_ds[i] = 1./(B_v0.dot(np.linalg.solve(K_ds, F_ext)))
    
    # Calculate Smith-Pippard number
    n_SP[i] = 2*bl.calc_Pn_rad(w)   # Multiply by 2 to account for both sides of the "hump"

In [ ]:
w.lace_cross(n_cross=3, n_spokes=24, diameter=1.8e-3, young_mod=200e9, offset=0.00)
w.apply_tension(0.01)

rr = bl.ModeMatrix(w, N=8*len(w.spokes))

K_sm = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=True)
K_ds = rr.calc_K_rim(buckling=False) + rr.calc_K_spk(smeared_spokes=False)


# Radial deflection under radial point load
th = np.linspace(-np.pi, np.pi, 4*len(w.spokes) + 1)

B_0 = rr.B_theta(0., comps=[0, 1])
B_u = rr.B_theta(th, comps=[0])
B_v = rr.B_theta(th, comps=[1])
B_w = rr.B_theta(th, comps=[2])

F_ext = rr.calc_F_ext([0.], np.array([[0., 1., 0., 0.]]))

d_rad_sm = np.linalg.solve(K_sm, F_ext)
d_rad_ds = np.linalg.solve(K_ds, F_ext)

# Lateral deflection under a drive torque

F_ext = rr.calc_F_ext([(1.5) * 2.*np.pi/len(w.spokes)], np.array([[0., 0., 1., 0.]]))

d_tan_sm = np.linalg.solve(K_sm, F_ext)
d_tan_ds = np.linalg.solve(K_ds, F_ext)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(6.5, 2.))


# Radial stiffness vs. Smith-Pippard number
ax[0].plot([0., 2.], [K_rad_sm[0]/1e6, K_rad_sm[0]/1e6], 'C0--')
ax[0].plot(n_SP, K_rad_ds / 1e6, 'C1*-')


ax[0].set_xlim([0., 3.0])
ax[0].set_xticks(np.arange(0., 3.1, 0.5))
ax[0].set_xticklabels(['0', '', '1', '', '2', '', '3'])
ax[0].set_xlabel('Spokes per char. len')

ax[0].set_ylim([0., 7.])
ax[0].set_ylabel(r'\$K_{rad}\$ [kN/mm]')


# Radial deflection for smeared and discrete spokes
ax[1].plot(B_v.dot(d_rad_sm) / B_0.dot(d_rad_ds)[1], 'C0--')
ax[1].plot(B_v.dot(d_rad_ds) / B_0.dot(d_rad_sm)[1], 'C1')

ax[1].set_xticks([]);
ax[1].set_xlabel(r'\$\theta\$')

ax[1].set_ylim([-0.25, 1.1])
ax[1].set_yticks([-0.25, 0., 0.25, 0.5, 0.75, 1.])
ax[1].set_yticklabels(['', '0', '', '', '', '1'])
ax[1].set_ylabel('Radial deflection')


# Radial deflection under tangential load
ax[2].plot(B_v.dot(d_tan_sm) / np.max(B_v.dot(d_tan_sm)), 'C0--')
ax[2].plot(B_v.dot(d_tan_ds) / np.max(B_v.dot(d_tan_sm)), 'C1')

ax[2].set_xticks([])
ax[2].set_xlabel(r'\$\theta\$')

ax[2].set_ylim([-6, 6])
ax[2].set_yticks([-5., 0., 5.])
ax[2].set_yticklabels(['-5', '0', '+5'])
ax[2].set_ylabel('Radial deflection')

plt.tight_layout()
plt.savefig('../figs/stress_analysis/_python_continuum_vs_discrete.pdf')